# **El patrón Repository**

In [3]:
!pip install sqlmodel

In [4]:
from sqlmodel import SQLModel, Field, Relationship
from typing import List, Optional

class BaseModel(SQLModel):
    id: Optional[int] = Field(default=None, primary_key=True)


class CategoryModel(BaseModel, table=True):
    __tablename__ = 'categories'

    name: str = Field(index=True, unique=True)

In [5]:
from typing import Optional

from pydantic import BaseModel, Field

class BaseSchema(BaseModel):
    class Config:
        from_attributes = True
        arbitrary_types_allowed = True

    id_key: Optional[int] = None

from typing import Optional, TYPE_CHECKING
class CategorySchema(BaseSchema):
    name: Optional[str] = None


In [7]:
from abc import abstractmethod, ABC
from typing import List, Type
from sqlmodel import Session

class BaseRepository(ABC):
    """
    BaseRepository is an abstract class that defines the methods
    that must be implemented by the repositories.
    """

    @property
    @abstractmethod
    def model(self) -> Type[BaseModel]:
        """
        SQLAlchemy model
        """

    @property
    @abstractmethod
    def schema(self) -> Type[BaseSchema]:
        """
        Pydantic schema
        """

    @abstractmethod
    def find_all(self) -> List[BaseSchema]:
        """
        Find all records
        :return: List[BaseSchema]
        """

    @abstractmethod
    def save(self, model: BaseModel) -> BaseSchema:
        """
        Save a record
        :param model: BaseModel
        :return: BaseSchema
        """

In [29]:
import logging
from contextlib import contextmanager
import pdb
from typing import Type, List
from sqlmodel import Session

class InstanceNotFoundError(Exception):
    """
    InstanceNotFoundError is raised when a record is not found
    """

class BaseRepositoryImpl(BaseRepository):
    """
    Class BaseRepositoryImpl implementa a BaseRepository
    """

    def __init__(self, model: Type[BaseModel], schema: Type[BaseSchema]):
        self._model = model
        self._schema = schema
        self.models= {}
        self.current_id = 1

    @property
    def model(self) -> Type[BaseModel]:
        return self._model

    @property
    def schema(self) -> Type[BaseSchema]:
        return self._schema


    def find_all(self) -> List[BaseSchema]:
            schemas = []
            for model in list(self.models.values()):
                schemas.append(self.schema.model_validate(model))
            return schemas

    def save(self, model: BaseModel) -> BaseSchema:
        self.models[self.current_id] = model
        self.current_id += 1
        return self.schema.model_validate(model)

In [9]:
class CategoryRepository(BaseRepositoryImpl):
    def __init__(self):
        super().__init__(CategoryModel, CategorySchema)

In [30]:
repo = BaseRepositoryImpl(CategoryModel, CategorySchema)

In [38]:
cat_model = CategoryModel()
cat_model.id=1
cat_model.name="test"

In [39]:
repo.save(cat_model)

CategorySchema(id_key=None, name='test')

In [40]:
repo.find_all()

[CategorySchema(id_key=None, name='test'),
 CategorySchema(id_key=None, name='test')]

In [41]:
list(repo.models.values())

[CategoryModel(id=None, name='test'), CategoryModel(id=1, name='test')]